# What is about ?

First look on cell cycle


(с) A.Chervov, A. Zinovyev

See paper: https://arxiv.org/abs/2208.05229

"Computational challenges of cell cycle analysis using single cell transcriptomics" Alexander Chervov, Andrei Zinovyev


Particular data:

Paper:
Roider, T., Seufert, J., Uvarovskii, A. et al. Dissecting intratumour heterogeneity of nodal B-cell lymphomas at the transcriptional, genetic and drug-response levels. Nat Cell Biol 22, 896–906 (2020). doi: https://doi.org/10.1038/s41556-020-0532-x

Data:
https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/VRJUNV




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install Import modules 

In [ ]:
!pip install scanpy
import scanpy as sc
import anndata

import time
t0start = time.time()

import pandas as pd
import numpy as np
import os
import sys

import matplotlib.pyplot as plt
#plt.style.use('dark_background')
import seaborn as sns

# Load Data

In [ ]:
# fn = '/kaggle/input/scrnaseq-b-cells-nature-immunology-2018-gse115795/FromPANGLAODB/GSM3190075_B_cells_SRA721637_SRS3416994.mat.txt'
# str_data_inf = ' B cells GSM3190075 '

# df = pd.read_csv(fn, sep = '\t' ) # , index_col = 0)
# df = df.T
# df

pathname = '/kaggle/input/scrnaseq-bt-cells-and-lymphomas-natcellbiol2020/tFL1/'
str_data_inf = ' Transformed Follicular Lymphoma 01 '
adata = sc.read_10x_mtx(pathname)
adata

# Normalize and Log

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Auxilliary function - phase plot

In [ ]:
G1S_genes_Tirosh = ['MCM5', 'PCNA', 'TYMS', 'FEN1', 'MCM2', 'MCM4', 'RRM1', 'UNG', 'GINS2', 'MCM6', 'CDCA7', 'DTL', 'PRIM1', 'UHRF1', 'MLF1IP', 'HELLS', 'RFC2', 'RPA2', 'NASP', 'RAD51AP1', 'GMNN', 'WDR76', 'SLBP', 'CCNE2', 'UBR7', 'POLD3', 'MSH2', 'ATAD2', 'RAD51', 'RRM2', 'CDC45', 'CDC6', 'EXO1', 'TIPIN', 'DSCC1', 'BLM', 'CASP8AP2', 'USP1', 'CLSPN', 'POLA1', 'CHAF1B', 'BRIP1', 'E2F8']
G2M_genes_Tirosh = ['HMGB2', 'CDK1', 'NUSAP1', 'UBE2C', 'BIRC5', 'TPX2', 'TOP2A', 'NDC80', 'CKS2', 'NUF2', 'CKS1B', 'MKI67', 'TMPO', 'CENPF', 'TACC3', 'FAM64A', 'SMC4', 'CCNB2', 'CKAP2L', 'CKAP2', 'AURKB', 'BUB1', 'KIF11', 'ANP32E', 'TUBB4B', 'GTSE1', 'KIF20B', 'HJURP', 'CDCA3', 'HN1', 'CDC20', 'TTK', 'CDC25C', 'KIF2C', 'RANGAP1', 'NCAPD2', 'DLGAP5', 'CDCA2', 'CDCA8', 'ECT2', 'KIF23', 'HMMR', 'AURKA', 'PSRC1', 'ANLN', 'LBR', 'CKAP5', 'CENPE', 'CTCF', 'NEK2', 'G2E3', 'GAS2L3', 'CBX5', 'CENPA']

# G1S_genes_Tirosh = ['ENSMUSG00000005410', 'ENSMUSG00000027342', 'ENSMUSG00000025747', 'ENSMUSG00000024742', 'ENSMUSG00000002870', 'ENSMUSG00000022673', 'ENSMUSG00000030978', 'ENSMUSG00000029591', 'ENSMUSG00000031821', 'ENSMUSG00000026355', 'ENSMUSG00000055612', 'ENSMUSG00000037474', 'ENSMUSG00000025395', 'ENSMUSG00000001228', 'ENSMUSG00000031629', 'ENSMUSG00000025001', 'ENSMUSG00000023104', 'ENSMUSG00000028884', 'ENSMUSG00000028693', 'ENSMUSG00000030346', 'ENSMUSG00000006715', 'ENSMUSG00000027242', 'ENSMUSG00000004642', 'ENSMUSG00000028212', 'ENSMUSG00000041712', 'ENSMUSG00000030726', 'ENSMUSG00000024151', 'ENSMUSG00000022360', 'ENSMUSG00000027323', 'ENSMUSG00000020649', 'ENSMUSG00000000028', 'ENSMUSG00000017499', 'ENSMUSG00000039748', 'ENSMUSG00000032397', 'ENSMUSG00000022422', 'ENSMUSG00000030528', 'ENSMUSG00000028282', 'ENSMUSG00000028560', 'ENSMUSG00000042489', 'ENSMUSG00000006678', 'ENSMUSG00000022945', 'ENSMUSG00000034329', 'ENSMUSG00000046179']
# G2M_genes_Tirosh = ['ENSMUSG00000054717', 'ENSMUSG00000019942', 'ENSMUSG00000027306', 'ENSMUSG00000001403', 'ENSMUSG00000017716', 'ENSMUSG00000027469', 'ENSMUSG00000020914', 'ENSMUSG00000024056', 'ENSMUSG00000062248', 'ENSMUSG00000026683', 'ENSMUSG00000028044', 'ENSMUSG00000031004', 'ENSMUSG00000019961', 'ENSMUSG00000026605', 'ENSMUSG00000037313', 'ENSMUSG00000020808', 'ENSMUSG00000034349', 'ENSMUSG00000032218', 'ENSMUSG00000048327', 'ENSMUSG00000037725', 'ENSMUSG00000020897', 'ENSMUSG00000027379', 'ENSMUSG00000012443', 'ENSMUSG00000015749', 'ENSMUSG00000036752', 'ENSMUSG00000022385', 'ENSMUSG00000024795', 'ENSMUSG00000044783', 'ENSMUSG00000023505', 'ENSMUSG00000020737', 'ENSMUSG00000006398', 'ENSMUSG00000038379', 'ENSMUSG00000044201', 'ENSMUSG00000028678', 'ENSMUSG00000022391', 'ENSMUSG00000038252', 'ENSMUSG00000037544', 'ENSMUSG00000048922', 'ENSMUSG00000028873', 'ENSMUSG00000027699', 'ENSMUSG00000032254', 'ENSMUSG00000020330', 'ENSMUSG00000027496', 'ENSMUSG00000068744', 'ENSMUSG00000036777', 'ENSMUSG00000004880', 'ENSMUSG00000040549', 'ENSMUSG00000045328', 'ENSMUSG00000005698', 'ENSMUSG00000026622', 'ENSMUSG00000035293', 'ENSMUSG00000074802', 'ENSMUSG00000009575', 'ENSMUSG00000029177']

def phase_plot(adata, list_genes1, list_genes2, mask = None, list_color_by = [None], str_suptitle = None, 
               str_title = None,
               xfigsize = 20, yfigsize = 10,
               xlabel = 'G1S score', ylabel = 'G2M score',
               n_x_subplots = 1, n_y_subplots = 1 #  #len(list_selected_cell_lines)
              ):

    if mask is None:
        mask = np.ones( adata.X.shape[0]).astype(bool) #  

    #list_color_by = ['total_counts']# , 'Cell cycle phase']

    
    total_count = 0
    for color_by_mode in  list_color_by: # , 'pct_counts_mt']:
        
        if n_x_subplots * n_y_subplots <= 1:
            fig = plt.figure(figsize = (xfigsize,yfigsize) ); c = 0
            if str_suptitle is not None:
                plt.suptitle(str_suptitle + ' N_cells= ' +str(mask.sum() ) + ' ' , fontsize = 20 )#' 
        else:
            if ( total_count % (n_x_subplots * n_y_subplots) ) == 0: 
                fig = plt.figure(figsize = (xfigsize,yfigsize) ); c = 0
                if str_suptitle is not None:
                    plt.suptitle(str_suptitle + ' N_cells= ' +str(mask.sum() ) + ' ' , fontsize = 20 )#' 
            c+=1; fig.add_subplot(n_y_subplots,n_x_subplots,c);
            
        total_count += 1
        
        
        color_by_field_name = color_by_mode
        color_by = None
        if color_by_mode is None:
            pass
        elif color_by_mode in adata.var.index:
            
            color_by = pd.Series( np.asarray(adata[mask,color_by_mode].X.sum(axis=1) ).ravel() , name = color_by_mode )
        elif color_by_mode in adata.obs:
            color_by = (adata[mask].obs[color_by_field_name])
        elif 'median_binarize_' in color_by_mode:
            color_by_field_name = color_by_mode[16:]
            if color_by_field_name in adata.obs:
                color_by = (adata[mask].obs[color_by_field_name]) > np.median( (adata[mask].obs[color_by_field_name]) )
        elif 'threshold_binarize_' in color_by_mode:
            color_by_field_name = color_by_mode.split('_')[3]
            threshold_binarize = float( color_by_mode.split('_')[2] )
            if color_by_field_name in adata.obs:
                color_by = (adata[mask].obs[color_by_field_name]) > threshold_binarize
        else:
            color_by = None

        ll = list_genes1
        I = np.where(adata[mask].var.index.isin(ll) > 0 )[0] # .sum()
        v1 = np.asarray(adata[mask].X[:,I].mean(axis = 1)).ravel()
        #print(len(I), v1.mean(), len(v1))    
        
        ll = list_genes2#  G2M_genes_Tirosh
        I = np.where(adata[mask].var.index.isin(ll) > 0 )[0] # .sum()
        #print(len(I))
        v2 = np.asarray(adata[mask].X[:,I].mean(axis = 1)).ravel()    


        if str_title is not None:
            plt.title( str_title  , fontsize = 20 )
        if color_by is None:
            ax = sns.scatterplot(x=v1, y = v2)# ,  hue= color_by,   alpha = 0.8, marker = '.')#, legend=None)
        else:
            #color_by = (adata.obs[color_by_field_name]) > np.median( adata.obs[color_by_field_name].values ) 
            ax = sns.scatterplot(x=v1, y = v2,  hue= color_by, palette = "rainbow")# 'viridis")# sns.color_palette("viridis", as_cmap=True),
                                #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
            plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
            plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title


        #plt.title(str_inf)# str_data_inf + ' ' + str_reducer + str_genes_inf + ' ' + col)
        plt.xlabel(xlabel , fontsize = 20)
        plt.ylabel(ylabel, fontsize = 20 )
    plt.show()         
        


# Phase plot

In [ ]:
adata.var

In [ ]:
str_g1s = '''ENSMUSG00000005410,
ENSMUSG00000027342,
ENSMUSG00000025747,
ENSMUSG00000024742,
ENSMUSG00000002870,
ENSMUSG00000022673,
ENSMUSG00000030978,
ENSMUSG00000029591,
ENSMUSG00000031821,
ENSMUSG00000026355,
ENSMUSG00000055612,
ENSMUSG00000037474,
ENSMUSG00000025395,
ENSMUSG00000001228,
ENSMUSG00000031629,
ENSMUSG00000025001,
ENSMUSG00000023104,
ENSMUSG00000028884,
ENSMUSG00000028693,
ENSMUSG00000030346,
ENSMUSG00000006715,
ENSMUSG00000027242,
ENSMUSG00000004642,
ENSMUSG00000028212,
ENSMUSG00000041712,
ENSMUSG00000030726,
ENSMUSG00000024151,
ENSMUSG00000022360,
ENSMUSG00000027323,
ENSMUSG00000020649,
ENSMUSG00000000028,
ENSMUSG00000017499,
ENSMUSG00000039748,
ENSMUSG00000032397,
ENSMUSG00000022422,
ENSMUSG00000030528,
ENSMUSG00000028282,
ENSMUSG00000028560,
ENSMUSG00000042489,
ENSMUSG00000006678,
ENSMUSG00000022945,
ENSMUSG00000034329,
ENSMUSG00000046179'''

str_g2m = '''ENSMUSG00000054717
ENSMUSG00000019942
ENSMUSG00000027306
ENSMUSG00000001403
ENSMUSG00000017716
ENSMUSG00000027469
ENSMUSG00000020914
ENSMUSG00000024056
ENSMUSG00000062248
ENSMUSG00000026683
ENSMUSG00000028044
ENSMUSG00000031004
ENSMUSG00000019961
ENSMUSG00000026605
ENSMUSG00000037313
ENSMUSG00000020808
ENSMUSG00000034349
ENSMUSG00000032218
ENSMUSG00000048327
ENSMUSG00000037725
ENSMUSG00000020897
ENSMUSG00000027379
ENSMUSG00000012443
ENSMUSG00000015749
ENSMUSG00000036752
ENSMUSG00000022385
ENSMUSG00000024795
ENSMUSG00000044783
ENSMUSG00000023505
ENSMUSG00000020737
ENSMUSG00000006398
ENSMUSG00000038379
ENSMUSG00000044201
ENSMUSG00000028678
ENSMUSG00000022391
ENSMUSG00000038252
ENSMUSG00000037544
ENSMUSG00000048922
ENSMUSG00000028873
ENSMUSG00000027699
ENSMUSG00000032254
ENSMUSG00000020330
ENSMUSG00000027496
ENSMUSG00000068744
ENSMUSG00000036777
ENSMUSG00000004880
ENSMUSG00000040549
ENSMUSG00000045328
ENSMUSG00000005698
ENSMUSG00000026622
ENSMUSG00000035293
ENSMUSG00000074802
ENSMUSG00000009575
ENSMUSG00000029177'''


In [ ]:
str_g1s = '''ENSG00000100297
ENSG00000132646
ENSG00000176890
ENSG00000168496
ENSG00000073111
ENSG00000104738
ENSG00000167325
ENSG00000076248
ENSG00000131153
ENSG00000076003
ENSG00000144354
ENSG00000143476
ENSG00000198056
ENSG00000276043
ENSG00000151725
ENSG00000119969
ENSG00000049541
ENSG00000117748
ENSG00000132780
ENSG00000111247
ENSG00000112312
ENSG00000092470
ENSG00000163950
ENSG00000175305
ENSG00000012963
ENSG00000077514
ENSG00000095002
ENSG00000156802
ENSG00000051180
ENSG00000171848
ENSG00000093009
ENSG00000094804
ENSG00000174371
ENSG00000075131
ENSG00000136982
ENSG00000197299
ENSG00000118412
ENSG00000162607
ENSG00000092853
ENSG00000101868
ENSG00000159259
ENSG00000136492
ENSG00000129173'''

str_g2m = '''ENSG00000164104
ENSG00000170312
ENSG00000137804
ENSG00000175063
ENSG00000089685
ENSG00000088325
ENSG00000131747
ENSG00000080986
ENSG00000123975
ENSG00000143228
ENSG00000173207
ENSG00000148773
ENSG00000120802
ENSG00000117724
ENSG00000013810
ENSG00000129195
ENSG00000113810
ENSG00000157456
ENSG00000169607
ENSG00000136108
ENSG00000178999
ENSG00000169679
ENSG00000138160
ENSG00000143401
ENSG00000188229
ENSG00000075218
ENSG00000138182
ENSG00000123485
ENSG00000111665
ENSG00000189159
ENSG00000117399
ENSG00000112742
ENSG00000158402
ENSG00000142945
ENSG00000100401
ENSG00000010292
ENSG00000126787
ENSG00000184661
ENSG00000134690
ENSG00000114346
ENSG00000137807
ENSG00000072571
ENSG00000087586
ENSG00000134222
ENSG00000011426
ENSG00000143815
ENSG00000175216
ENSG00000138778
ENSG00000102974
ENSG00000117650
ENSG00000092140
ENSG00000139354
ENSG00000094916
ENSG00000115163'''

In [ ]:
list2 = str_g2m.split('\n')# ,'').split(',')
print(list2)

In [ ]:
#list1 = str_g1s.replace('\n','').split(',')
list1 = str_g1s.split('\n')# ,'').split(',')

print(list1)

In [ ]:
phase_plot(adata, list_genes1 =G1S_genes_Tirosh , # list1, #  
           list_genes2= G2M_genes_Tirosh, # list2, # 
           mask = None,str_title = 'Tirosh genes' ,
               str_suptitle = str_data_inf + ' n_cells = ' + str( adata.shape[0] ),
          list_color_by = ['Cell Type',   'CCNB1','CCNE1'],)

In [ ]:
nan

In [ ]:
nan